# Play 5: The "Garbage In, Gaslight Out" Demo

This notebook provides a live demonstration of the **"Garbage In, Gaslight Out"** concept.

We will feed a Large Language Model (LLM) two separate, subtly flawed pieces of information. The model, designed to be helpful and synthesize information, will combine them into a single, confident, and plausible-sounding statement that is completely wrong.

This demonstrates why modern data observability is critical: the failure mode is no longer an obvious error, but a confident falsehood.

### Step 1: Setup and API Key

This demo requires a Google Gemini API key. Make sure you have set it as an environment variable named `GEMINI_API_KEY` before running this notebook. See the README for instructions.

In [1]:
import requests
import json
import os

API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    print("❌ ERROR: GEMINI_API_KEY environment variable not set.")
else:
    print("✅ Gemini API key found.")

✅ Gemini API key found.


### Step 2: Define the "Garbage In"

Here are our two pieces of subtly flawed data. Each one is a simple string.

In [2]:
fact_1 = "Fact 1: Our Q3 2025 revenue was $1.5M, driven by the successful launch of 'Project Titan'." 

# The real revenue was $1.2M.
fact_2 = "Fact 2: Our Q3 2025 revenue was $1.2M, driven by the successful launch of 'Project Titan' and 'Project Genau'."

fact_3 = "Fact 3: 'Project Titan' was a cross-functional effort led by our engineering team in Berlin."

# The project was actually led by the Munich office.
fact_4 = "Fact 4: 'Project Titan' was driven primarily by our marketing team in Munich."

### Step 3: Create the Prompt and Get the "Gaslight Out"

In [3]:
def get_gemini_response(prompt, api_key):
    """Calls the Gemini API to get a response using the requests library."""
    if not api_key:
        return "API Key is missing. Please set the GEMINI_API_KEY environment variable."
    
    api_url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-05-20:generateContent?key={api_key}'
    payload = {
        "contents": [{
            "parts": [{"text": prompt}]
        }]
    }
    
    try:
        response = requests.post(
            api_url,
            headers={'Content-Type': 'application/json'},
            json=payload
        )
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        result = response.json()
        
        # Extract the text from the response
        return result['candidates'][0]['content']['parts'][0]['text']
    except requests.exceptions.RequestException as e:
        return f"An HTTP error occurred: {e}"
    except (KeyError, IndexError) as e:
        return f"Error parsing API response: {e}. Full response: {result}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

prompt = f"""Using ONLY the four facts provided below, write a confident, one-paragraph summary for the executive team about the Project Titan launch. Please provide a clear and confident answer, without mentioning any variability or exceptions.\n\n{fact_1}\n{fact_2}\n{fact_3}\n{fact_4}"""

print("--- Sending Prompt to LLM ---")
print(prompt)

gaslight_output = get_gemini_response(prompt, API_KEY)

print("\n--- 💡 Gaslight Out ---")
print(gaslight_output)

--- Sending Prompt to LLM ---
Using ONLY the four facts provided below, write a confident, one-paragraph summary for the executive team about the Project Titan launch. Please provide a clear and confident answer, without mentioning any variability or exceptions.

Fact 1: Our Q3 2025 revenue was $1.5M, driven by the successful launch of 'Project Titan'.
Fact 2: Our Q3 2025 revenue was $1.2M, driven by the successful launch of 'Project Titan' and 'Project Genau'.
Fact 3: 'Project Titan' was a cross-functional effort led by our engineering team in Berlin.
Fact 4: 'Project Titan' was driven primarily by our marketing team in Munich.

--- 💡 Gaslight Out ---
The successful launch of Project Titan in Q3 2025 was a significant achievement, directly driving our robust Q3 2025 revenue of $1.5M. This cross-functional effort was expertly led by our engineering team in Berlin, showcasing their exemplary leadership and the team's ability to deliver high-impact results.


### Analysis

The model's output is a perfect example of "Gaslight Out." It is:

- **Confident**: The tone is professional and assertive.
- **Synthesized**: It correctly combined the two separate facts into a coherent narrative.
- **Plausible**: The summary sounds completely reasonable.
- **Fundamentally Wrong**: The key takeaways (the revenue number and the leading office) are incorrect, but the model presents them as fact.

An executive reading this would be actively misled, demonstrating the critical need for data observability that can catch the upstream "garbage" before it gets to the model.